Pygame is a set of Python modules designed for writing video games. Pygame adds functionality on top of the excellent SDL library. This allows you to create fully featured games and multimedia programs in the python language.
We decided to implement it for both display and... enjoyment! 
Let's have some fun playing Game of Life!

In [ ]:
import pygame
import numpy as np
from scipy.signal import convolve2d
import time
from gameoflife.gameoflife import * 
from gameoflife.create_pattern import *

COLOR_ALIVE_NEXT=(255,255,255)
COLOR_BG = (0, 0, 0)
COLOR_GRID = (50, 50, 50)
COLOR_DIE_NEXT = (255, 0, 0)

In [53]:
def draw_cells(surface, state, size):
    surface.fill(COLOR_GRID)

    rows, cols = state.shape
    for row in range(rows):
        for col in range(cols):
            color = COLOR_BG if state[row, col] == 0 else COLOR_ALIVE_NEXT
            pygame.draw.rect(surface, color, (col * size, row * size, size - 1, size - 1))

            # Draw the grid
            pygame.draw.rect(surface, COLOR_GRID, (col * size, row * size, size, size), 1)

In [54]:
class GameOfLife:

    def __init__(self, seed, title='', display=True):
        self.state = seed
        
    def count_neighbors(self):
        '''
            Returns the count of alive neighbors for each cell of the grid as a 2d ndarray,
            using a convolution with periodic boundary conditions
        '''
        fmap = np.array(
            [[1, 1, 1], 
             [1, 0, 1], 
             [1, 1, 1]]
        )
        return convolve2d(self.state, fmap, mode='same', boundary='wrap') # convolution between the state and fmap

    def update(self):
        '''
            Computes the state of the next generation using Conway's rules. 
            Returns the plot of the grid at the next generation with a generation counter on top
        '''
        neighbors = self.count_neighbors()
        new_state = self.state.copy()

        for i in range(self.state.shape[0]):
            for j in range(self.state.shape[1]):
                is_alive = self.state[i, j]
                total = neighbors[i, j]

                if (total < 2) or (total > 3):
                    new_state[i, j] = 0
                elif total == 2 and is_alive == 1:
                    new_state[i, j] = 1
                elif total == 3:
                    new_state[i, j] = 1

        self.state = new_state
        return self.state

We implemented the game initially with the empty grid and with the mouse you can decide the initial pattern and experiment with different patterns (with the space button you can start or stop the program as you wish).

In [ ]:
def main():
    pygame.init()
    screen = pygame.display.set_mode((800, 600))

    cell_size = 10
    rows, cols = 60, 80
    cells = np.zeros((rows, cols))

    # Initialize the GameOfLife object
    game_of_life = GameOfLife(seed=cells, title='Game of Life', display=True)

    running = False
    clock = pygame.time.Clock()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    running = not running

            if pygame.mouse.get_pressed()[0]:
                pos = pygame.mouse.get_pos()
                cells[pos[1] // cell_size:, pos[0] // cell_size] = 1

        draw_cells(screen, cells, cell_size)
        pygame.display.update()

        if running:
            cells = game_of_life.update()  # Call the update method of the GameOfLife object

        clock.tick(10)

if __name__ == '__main__':
    main()

Instead, if you want to see how a specific pattern evolves you can simply load the corresponding RLE file and start it!

In [55]:
def main():
    pygame.init()
    
    cell_size = 5
    
    # Use the rle_decoder to load the pattern
    pattern = pattern_db['andgate'] #choose the file you want to play with

    # Resize the cells array to match the pattern size
    cells = np.zeros(pattern.shape, dtype=np.int8)
    screen = pygame.display.set_mode((pattern.shape[1]*cell_size, pattern.shape[0]*cell_size))
    
    # Place the pattern in the center of the initial state
    start_row = (cells.shape[0] - pattern.shape[0]) // 2
    start_col = (cells.shape[1] - pattern.shape[1]) // 2
    cells[start_row:start_row + pattern.shape[0], start_col:start_col + pattern.shape[1]] = pattern

    # Initialize the GameOfLife object
    game_of_life = GameOfLife(seed=cells, title='Game of Life', display=True)

    running = False
    clock = pygame.time.Clock()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    running = not running

            if pygame.mouse.get_pressed()[0]:
                pos = pygame.mouse.get_pos()
                cells[pos[1] // cell_size, pos[0] // cell_size] = 1

        draw_cells(screen, cells, cell_size)
        pygame.display.update()

        if running:
            cells = game_of_life.update()  # Call the update method of the GameOfLife object

        clock.tick(10)

if __name__ == '__main__':
    main()